In [ ]:
import os
import tarfile
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

# Path to the dataset tar.gz file
dataset_path = '/content/dev-clean.tar.gz'
# Directory to extract the files
extracted_dir = '/content/extracted'
# Directory to save spectrogram images
spectrogram_dir = '/content/mel_spectrograms'

# Create directories if they don't exist
os.makedirs(extracted_dir, exist_ok=True)
os.makedirs(spectrogram_dir, exist_ok=True)

# Step 1: Extract the dataset
try:
    with tarfile.open(dataset_path, 'r:gz') as tar:
        tar.extractall(path=extracted_dir)
        print(f"Extraction completed successfully to {extracted_dir}")
except tarfile.ReadError:
    print("Error: Unable to read the tar file. It might be corrupted.")
except EOFError:
    print("Error: The file seems to be incomplete or corrupted.")

# Step 2: Function to generate and save Mel spectrogram
def save_mel_spectrogram(audio_path, save_path, noise_factor=0.005):
    """
    Generate and save a Mel spectrogram as an image.
    Optionally augment the audio with noise.

    Parameters:
    audio_path (str): Path to the input audio file.
    save_path (str): Path to save the Mel spectrogram image.
    noise_factor (float): Factor controlling the amount of noise for augmentation.
    """
    try:
        # Load the audio file
        y, sr = librosa.load(audio_path, sr=16000)  # Using 16kHz sample rate

        # Augment the audio by adding noise
        noise = np.random.randn(len(y))
        augmented_audio = y + noise_factor * noise

        # Generate Mel spectrogram
        S = librosa.feature.melspectrogram(y=augmented_audio, sr=sr, n_mels=128, fmax=8000)
        S_dB = librosa.power_to_db(S, ref=np.max)

        # Save Mel spectrogram as image
        plt.figure(figsize=(10, 4))
        librosa.display.specshow(S_dB, sr=sr, x_axis='time', y_axis='mel')
        plt.colorbar(format='%+2.0f dB')
        plt.title('Mel spectrogram')
        plt.tight_layout()

        # Ensure the directory exists
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        plt.savefig(save_path)
        plt.close()
        print(f"Saved Mel spectrogram at {save_path}")
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")

# Step 3: Process each .flac file in the extracted dataset
def process_dataset(dataset_folder, output_folder, noise_factor=0.005):
    """
    Process each .flac file in a dataset folder, generate Mel spectrograms, and save them as images.

    Parameters:
    dataset_folder (str): Path to the folder containing .flac files.
    output_folder (str): Path to save the spectrogram images.
    noise_factor (float): Factor controlling the amount of noise for augmentation.
    """
    for root, _, files in os.walk(dataset_folder):
        for file in files:
            if file.endswith(".flac"):
                # Input and output paths
                audio_path = os.path.join(root, file)
                relative_path = os.path.relpath(root, dataset_folder)
                spectrogram_save_path = os.path.join(output_folder, relative_path, f"{os.path.splitext(file)[0]}.png")

                # Process and save the spectrogram
                save_mel_spectrogram(audio_path, spectrogram_save_path, noise_factor)

# Example usage
process_dataset(extracted_dir, spectrogram_dir)


In [ ]:
import librosa
import numpy as np
import soundfile as sf
import os

# Define directories for input and output
extracted_dir = '/content/extracted'  # The directory containing the raw audio files
noisy_audio_dir = '/content/noisy_audio'  # Directory to save noisy audio files
clean_audio_dir = '/content/clean_audio'  # Directory to save clean audio files

# Create directories if they don't exist
os.makedirs(noisy_audio_dir, exist_ok=True)
os.makedirs(clean_audio_dir, exist_ok=True)

# Function to generate and save noisy/clean audio
def save_noisy_and_clean_audio(audio_path, clean_save_path, noisy_save_path, noise_factor=0.005):
    try:
        # Load the clean audio file
        y, sr = librosa.load(audio_path, sr=16000)  # Using 16kHz sample rate

        # Save clean audio in .flac format
        sf.write(clean_save_path, y, sr)
        print(f"Saved clean audio at {clean_save_path}")

        # Augment the audio by adding noise
        noise = np.random.randn(len(y))  # White noise
        augmented_audio = y + noise_factor * noise  # Add noise to the clean audio

        # Save noisy audio in .flac format
        sf.write(noisy_save_path, augmented_audio, sr)
        print(f"Saved noisy audio at {noisy_save_path}")
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")

# Loop through all audio files in the extracted directory and process them
found_files = 0  # To track if any files are found
for root, dirs, files in os.walk(extracted_dir):
    for file in files:
        if file.endswith(".flac"):  # Process only .flac files
            found_files += 1
            audio_file = os.path.join(root, file)

            # Define paths for saving clean and noisy audio
            clean_audio_path = os.path.join(clean_audio_dir, os.path.basename(file).replace('.flac', '_clean.flac'))
            noisy_audio_path = os.path.join(noisy_audio_dir, os.path.basename(file).replace('.flac', '_noisy.flac'))

            # Generate clean and noisy audio
            save_noisy_and_clean_audio(audio_file, clean_audio_path, noisy_audio_path)

if found_files == 0:
    print("No .flac files found in the extracted directory.")
else:
    print(f"Processed {found_files} .flac files.")


In [ ]:
import librosa
import numpy as np
import soundfile as sf
import os

# Define directories for input and output
extracted_dir = '/content/extracted'  # The directory containing the raw audio files
noisy_audio_dir = '/content/noisy_audio'  # Directory to save noisy audio files
clean_audio_dir = '/content/clean_audio'  # Directory to save clean audio files

# Target length in samples (5 seconds at 16 kHz)
target_length = 16000 * 5  # 5 seconds of audio at 16kHz sample rate

# Create directories if they don't exist
os.makedirs(noisy_audio_dir, exist_ok=True)
os.makedirs(clean_audio_dir, exist_ok=True)

# Function to generate and save noisy/clean audio with standard size
def save_noisy_and_clean_audio(audio_path, clean_save_path, noisy_save_path, noise_factor=0.005):
    try:
        # Load the clean audio file
        y, sr = librosa.load(audio_path, sr=16000)  # Using 16kHz sample rate

        # Trim or pad the audio to the target length
        if len(y) > target_length:
            y = y[:target_length]  # Trim if longer
        elif len(y) < target_length:
            y = np.pad(y, (0, target_length - len(y)), mode='constant')  # Pad if shorter

        # Save clean audio in .flac format
        sf.write(clean_save_path, y, sr)
        print(f"Saved clean audio at {clean_save_path}")

        # Augment the audio by adding noise
        noise = np.random.randn(len(y))  # White noise
        augmented_audio = y + noise_factor * noise  # Add noise to the clean audio

        # Save noisy audio in .flac format
        sf.write(noisy_save_path, augmented_audio, sr)
        print(f"Saved noisy audio at {noisy_save_path}")
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")

# Loop through all audio files in the extracted directory and process them
found_files = 0  # To track if any files are found
for root, dirs, files in os.walk(extracted_dir):
    for file in files:
        if file.endswith(".flac"):  # Process only .flac files
            found_files += 1
            audio_file = os.path.join(root, file)

            # Define paths for saving clean and noisy audio
            clean_audio_path = os.path.join(clean_audio_dir, os.path.basename(file).replace('.flac', '_clean.flac'))
            noisy_audio_path = os.path.join(noisy_audio_dir, os.path.basename(file).replace('.flac', '_noisy.flac'))

            # Generate clean and noisy audio with standard size
            save_noisy_and_clean_audio(audio_file, clean_audio_path, noisy_audio_path)

if found_files == 0:
    print("No .flac files found in the extracted directory.")
else:
    print(f"Processed {found_files} .flac files.")
